In [ ]:
# MS 434 Supply Chain Management Lab 3

# Group Number: 10
# Student Name: Manjary Muruganandan, Achchala Deepan 
# Student ID: 20950662, 20943939 

In [ ]:
# Practice 1

import pandas as pd

def summarize_course(grades):
    """
    Calculate average, standard deviation, max, and min for a list of grades.

    Parameters:
        grades (list): List of numerical grades.

    Returns:
        pd.Series: A summary with avg, std, max, and min.
    """
    
    return pd.Series({
        'Average': sum(grades) / len(grades),
        'Standard Deviation': pd.Series(grades).std(),
        'Max': max(grades),
        'Min': min(grades)
    })

grades_MS251 = [78, 85, 92, 88, 76, 81, 90]
summary_MS251 = summarize_course(grades_MS251)
print("MS251 Summary:")
print(summary_MS251)

grades_MS434 = [84, 79, 91, 88, 75, 80, 85, 89]
summary_MS434 = summarize_course(grades_MS434)
print("MS434 Summary:")
print(summary_MS434)

grades_MS452 = [82, 87, 90, 85, 78, 80, 76, 88, 92]
summary_MS452 = summarize_course(grades_MS452)
print("MS452 Summary:")
print(summary_MS452)


In [3]:
# Demonstration of "class", you don't need to do anything

class UserGymProfile:
    def __init__(self, name, age, gender, weight):
        self.name = name
        self.age = age
        self.gender = gender.lower()
        self.weight = weight

        # Advanced attributes: These will be set by run()
        self.age_group = None
        self.weight_category = None
        self.intensity = None
        self.schedule = {}

    def run(self):

        """Main controller: decide classification, intensity, schedule"""

        # Step 1: Basic functions: classify age and weight
        self.age_group = self.classify_age_group()
        self.weight_category = self.classify_weight_category()

        # Step 2: Analysis: decide intensity based on rules
        self.intensity = self.recommend_intensity()

        # Step 3: generate weekly schedule
        self.schedule = self.generate_schedule()

    def classify_age_group(self):

        if self.age < 18:
            return "teen"
        elif self.age <= 40:
            return "adult"
        else:
            return "senior"

    def classify_weight_category(self):

        if self.weight < 50:
            return "underweight"
        elif 50 <= self.weight <= 80:
            return "normal"
        else:
            return "overweight"

    def recommend_intensity(self):
        # Rule 1: Female and age ≥ 50 → generally lower intensity for joint and bone care
        if self.gender == "female" and self.age >= 50:
            return "low" if self.weight_category != "normal" else "moderate"

        # Rule 2: Teenagers
        if self.age_group == "teen":
            return "moderate" if self.weight_category == "normal" else "low"

        # Rule 3: Adults
        if self.age_group == "adult":
            if self.weight_category == "normal":
                return "high"
            elif self.weight_category == "underweight":
                return "moderate"
            else:  # overweight
                return "moderate"

        # Rule 4: Seniors
        if self.age_group == "senior":
            if self.weight_category == "normal":
                return "moderate"
            else:
                return "low"

        # Fallback
        return "moderate"

    def generate_schedule(self):
        if self.intensity == "low":
            return {
                "Monday": "Gentle walking or elliptical (30 min)",
                "Wednesday": "Light stretching / Yoga (30 min)",
                "Friday": "Chair-based exercises or swimming (30 min)",
            }
        elif self.intensity == "moderate":
            return {
                "Monday": "Cardio (bike or treadmill, 45 min)",
                "Wednesday": "Strength training (machines, 30 min)",
                "Friday": "Low-impact HIIT or circuit (30 min)",
            }
        else:  # high
            return {
                "Monday": "Strength training (upper body, 60 min)",
                "Tuesday": "Cardio + core (45 min)",
                "Thursday": "Strength training (lower body, 60 min)",
                "Saturday": "Full body HIIT (30 min)",
            }

    def show_summary(self):
        print(f"=== {self.name}'s Gym Plan ===")
        print(f"Age: {self.age} ({self.age_group})")
        print(f"Gender: {self.gender}")
        print(f"Weight: {self.weight}kg ({self.weight_category})")
        print(f"Recommended Intensity: {self.intensity}")
        print("Weekly Schedule:")
        for day, workout in self.schedule.items():
            print(f"  {day}: {workout}")

In [ ]:
user1 = UserGymProfile(name = "Lily", age = 55, gender = "female", weight= 62)
user1.run()            # Runs logic and sets attributes
user1.show_summary()   # Displays results

user2 = UserGymProfile(name="George", age=72, gender="male", weight=82)
user2.run()            # Runs logic and sets attributes
user2.show_summary()   # Displays results

user3 = UserGymProfile(name="Ava", age=28, gender="female", weight=59)
user3.run()            # Runs logic and sets attributes
user3.show_summary()   # Displays results

# please test more cases on your own!

In [ ]:
from docplex.mp.model import Model

class FlexibleProductMixOptimizer:
    def __init__(self, products, profits, usage, limits):
        """
        products: list of product names (e.g., ["A", "B", "C"])
        profits: dict of profit per unit (e.g., {"A": 30, "B": 20, "C": 40})
        usage: dict of resource usage (e.g., {"Machine": {"A": 2, ...}})
        limits: dict of total available resources (e.g., {"Machine": 100})
        """
        self.products = products
        self.profits = profits
        self.usage = usage
        self.limits = limits
        self.model = Model(name = "ProductMix")

    def run(self):
        self.build_model()
        return self.solve()

    def build_model(self):
        # Step 1: Define decision variables for each product
        self.units = self.model.continuous_var_dict(
            self.products, lb=0, name="Units"
        )

        # Step 2: Add constraints for each resource
        for r, cap in self.limits.items():
            self.model.add_constraint(
                self.model.sum(
                    self.usage[r].get(p, 0) * self.units[p] for p in self.products
                )
                <= cap,
                ctname=f"Cap_{r}",
            )

        # Step 3: Define the objective function (maximize total profit)
        self.model.maximize(
            self.model.sum(self.profits[p] * self.units[p] for p in self.products)
        )

    def solve(self):
        solution = self.model.solve()
        if solution:
            print("\nOptimal Production Plan")
            print("-----------------------")
            plan = {p: self.units[p].solution_value for p in self.products}
            for p, qty in plan.items():
                print(f"{p}: {qty:.2f} units")
            print(f"\nTotal Profit: {sol.objective_value:.2f}")
           
            return 
        else:
            print("\nNo feasible solution found.")
            return None



In [ ]:
# products = ["A", "B"]
# profits = {"A": 30, "B": 35}
# usage = {"Machine": {"A": 1, "B": 2}, "Labor": {"A": 2, "B": 1}}
# limits = {"Machine": 100, "Labor": 80}

# products = ["A", "B", "C"]
# profits = {"A": 30, "B": 35, "C": 60}
# usage = {"Machine": {"A": 1, "B": 2, "C": 3}, "Labor": {"A": 2, "B": 1, "C": 2}}
# limits = {"Machine": 100, "Labor": 80}

products = ["A", "B", "C", "D"]
profits = {"A": 30, "B": 35, "C": 60, "D": 25}
usage = {"Machine": {"A": 1, "B": 2, "C": 3, "D": 1.5}, "Labor": {"A": 2, "B": 1, "C": 2, "D": 2}}
limits = { "Machine": 100, "Labor": 80}

optimizer = FlexibleProductMixOptimizer(products, profits, usage, limits)
result = optimizer.run()
# print(result)

In [ ]:
# Heuristic for the VRP: Nearest neighber algorithm

import math
import pandas as pd
import numpy as np

# ----------------------------
# 1. Load or define the data
# ----------------------------

# Sample data from the lecture slide
# Format: Node, x, y, demand
data = [
    (0, 5, 5, 0),
    (1, 5, 8, 3),
    (2, 6, 7, 4),
    (3, 7, 9, 2),
    (4, 8, 5, 6),
    (5, 7, 3, 5),
    (6, 5, 3, 3),
    (7, 3, 2, 4),
    (8, 2, 5, 6)]
df = pd.DataFrame(data, columns = ['node', 'x', 'y', 'demand'])

Q = 15  # truck capacity

# ----------------------------
# 2. Utility Functions
# ----------------------------

def calculate_distance_matrix(df):
    n = len(df)
    dist = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            # it is called Manhattan distance
            dist[i][j] = abs(df.loc[i, 'x'] - df.loc[j, 'x']) + abs(df.loc[i, 'y'] - df.loc[j, 'y']) 
    df_dist = pd.DataFrame(dist)
    return df_dist

# ----------------------------
# 3. Nearest Neighbor Algorithm
# ----------------------------

def nearest_neighbor(df, dist_matrix, capacity_limit):
    depot = 0
    unvisited = set(df['node'])
    unvisited.remove(depot)      
    routes = []
    total_distance = 0.0

    while unvisited:
        route = [depot]
        remaining_capacity = capacity_limit
        current = depot

        while True:
            feasible = [n for n in unvisited
                        if df.loc[df['node'] == n, 'demand'].iat[0] <= remaining_capacity]
            if not feasible:
                break

            next_node = min(feasible, key=lambda n: dist_matrix.iat[current, n])

            route.append(next_node)
            total_distance += dist_matrix.iat[current, next_node]
            remaining_capacity -= df.loc[df['node'] == next_node, 'demand'].iat[0]
            current = next_node
            unvisited.remove(next_node)

        total_distance += dist_matrix.iat[current, depot]
        route.append(depot)
        routes.append(route)

    return routes, total_distance


    """
    Nearest Neighbor Heuristic for Capacitated VRP (Vehicle Routing Problem)
    
    Parameters:
        df: DataFrame with columns ['node', 'x', 'y', 'demand']
        dist_matrix: Precomputed distance matrix (e.g., Manhattan or Euclidean)
        capacity_limit: Vehicle capacity constraint
        
    Returns:
        routes: List of routes (each route is a list of node IDs)
        total_distance: Total distance traveled across all routes
    """
    



    return 

# ----------------------------
# 4. Run the Algorithm
# ----------------------------

dist_matrix = calculate_distance_matrix(df)
display(dist_matrix)
routes, total_dist = nearest_neighbor(df, dist_matrix, Q)

# ----------------------------
# 5. Display Results
# ----------------------------

for i, r in enumerate(routes, 1):
    print(f"Route {i}: {' - '.join(map(str, r))}")
print(f"Total distance: {total_dist:.2f}")


In [ ]:
#Lab Assignment 3

import math
import pandas as pd
import numpy as np

 
class NearestNeighborAlgorithm:

    # ------------------------------------------------------------------
    # 0. Constructor & validation
    # ------------------------------------------------------------------
    def __init__(self, excel_file, distance="manhattan", capacity_limit=15):
        self.df = pd.read_excel(excel_file)
        self.distance = distance.lower()
        self.capacity_limit = float(capacity_limit)

        self._check_format()
        self.dist_matrix = self._calc_distance_matrix()
        self.routes, self.total_distance = self._nearest_neighbor()

    # ------------------------------------------------------------------
    # 1. Quick input sanity checks
    # ------------------------------------------------------------------
    def _check_format(self):
        required = {"node", "x", "y", "demand"}
        if not required.issubset(set(self.df.columns.str.lower())):
            raise ValueError(
                f"Excel file must contain columns {required}. "
                f"Found {set(self.df.columns)}"
            )
        if 0 not in self.df["node"].values:
            raise ValueError("Node 0 (the depot) must be present in the data.")
        if self.df["demand"].lt(0).any():
            raise ValueError("All demands must be non‑negative.")

    # ------------------------------------------------------------------
    # 2. Build the distance matrix
    # ------------------------------------------------------------------
    def _calc_distance_matrix(self):
        n = len(self.df)
        x, y = self.df["x"].values, self.df["y"].values
        dist = np.zeros((n, n))

        if self.distance == "manhattan":
            for i in range(n):
                for j in range(n):
                    dist[i, j] = abs(x[i] - x[j]) + abs(y[i] - y[j])
        elif self.distance == "euclidean":
            for i in range(n):
                for j in range(n):
                    dist[i, j] = math.hypot(x[i] - x[j], y[i] - y[j])
        else:
            raise ValueError("distance must be 'manhattan' or 'euclidean'")

        return pd.DataFrame(dist, index=self.df["node"], columns=self.df["node"])

    # ------------------------------------------------------------------
    # 3. Nearest‑neighbor heuristic
    # ------------------------------------------------------------------
    def _nearest_neighbor(self):
        depot = 0
        unvisited = set(self.df["node"]) - {depot}

        routes, grand_total = [], 0.0
        demand_lookup = dict(zip(self.df["node"], self.df["demand"]))

        while unvisited:
            route, load = [depot], 0
            current = depot
            remaining = self.capacity_limit

            while True:
                # pick feasible customers
                feasible = [
                    n
                    for n in unvisited
                    if demand_lookup[n] <= remaining
                ]
                if not feasible:
                    break

                # choose the nearest feasible customer
                next_node = min(
                    feasible, key=lambda n: self.dist_matrix.at[current, n]
                )

                # update state
                route.append(next_node)
                grand_total += self.dist_matrix.at[current, next_node]
                remaining -= demand_lookup[next_node]
                load += demand_lookup[next_node]
                current = next_node
                unvisited.remove(next_node)

            # close the tour
            grand_total += self.dist_matrix.at[current, depot]
            route.append(depot)
            routes.append(
                {
                    "route": route,
                    "load": load,
                    "distance": self._route_length(route),
                }
            )

        return routes, grand_total

    def _route_length(self, route):
        """Helper to compute length for one (closed) route."""
        length = 0.0
        for i in range(len(route) - 1):
            length += self.dist_matrix.at[route[i], route[i + 1]]
        return length

    # ------------------------------------------------------------------
    # 4. Public helpers
    # ------------------------------------------------------------------
    def run(self):
        """Return a tidy DataFrame with every route and a footer row."""
        rows = []
        for idx, r in enumerate(self.routes, 1):
            rows.append(
                {
                    "Trip": idx,
                    "Sequence": " - ".join(map(str, r["route"])),
                    "Load": r["load"],
                    "Distance": round(r["distance"], 2),
                }
            )
        summary = pd.DataFrame(rows)
        footer = pd.DataFrame(
            {
                "Trip": ["TOTAL"],
                "Sequence": [""],
                "Load": [summary["Load"].sum()],
                "Distance": [round(self.total_distance, 2)],
            }
        )
        return pd.concat([summary, footer], ignore_index=True)

    def __repr__(self):
        return f"<NearestNeighborAlgorithm {len(self.routes)} routes, {self.total_distance:.2f} total distance>"
